In [2]:
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
import matplotlib.pylab as pylab
import pickle
from tqdm import tqdm
from tqdm.contrib import itertools
from scipy.ndimage import gaussian_filter1d

params = {'legend.fontsize': 10,
          'figure.figsize': (5.728, 3.54),
          'figure.dpi': 100,
          'axes.labelsize': 15,
          'axes.titlesize': 15,
          'xtick.labelsize': 10,
          'ytick.labelsize': 10}

pylab.rcParams.update(params)

colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', '#FFDB58', '#a65628']
markers = ['o', "^", "v", "*", "+", "x", "s", "p"]
linestyles = ['-', '--', '-.', ':', '-', '--', ':', '-.']

In [3]:
class MotionPrimitiveNeuron():
    NeuronState = namedtuple('NeuronState', ['V', 'w', 'spk', 'I'])

    def __init__(self, parameters):
        super(MotionPrimitiveNeuron, self).__init__()
        self.tau = parameters['tau']
        self.V_R = parameters['V_R']
        self.V_T = parameters['V_T']
        self.w = parameters['w']
        self.n = parameters['n']
        self.N_input = parameters['N_input']
        self.dt = parameters['dt']
        self.state = None

    def initialize_state(self):
        self.state = None

    def forward(self, input):
        if self.state is None:
            self.state = self.NeuronState(V=np.full((self.n,), self.V_R),
                                          w=np.array(self.w),
                                          spk=np.zeros(self.n),
                                          I=np.zeros((self.n, self.N_input)))
            
        V = self.state.V
        w = self.state.w
        I = self.state.I

        V += self.dt*(self.V_R-V)/self.tau
        V += np.sum(w*input, axis=-1)

        spk = np.heaviside(V - self.V_T, 0)
        V = (1 - spk) * V + spk * self.V_R + (1 - spk)

        self.state = self.NeuronState(V=V, w=w, spk=spk, I=I)

        return V, spk

In [3]:
with open('data/spike_position', 'rb') as file:
    spike_position = pickle.load(file)
    
with open('data/spike_motion', 'rb') as file:
    spike_motion = pickle.load(file) 
    
N_SIMULATIONS = 1
N_HAIRS = 50    
N_ANGLES = 18
T_TOTAL = 5
dt = 0.001

N_STEPS = int(T_TOTAL/dt)
time = np.linspace(0, T_TOTAL, num=N_STEPS)

primitive_parameters = {'tau': 1.5e-3, 'V_T': -50e-3, 'V_R': -70e-3, 'n': 0, 'w': 0,
                    'N_input': 3, 'dt': dt, 'refrac': 0}

primitive_neuron = MotionPrimitiveNeuron(primitive_parameters)
primitive_neuron.initialize_state()

In [ ]:
for k in tqdm(range(N_SIMULATIONS), desc='Network progress'):        
    for i in range(N_STEPS):
        _, spike_primitive[i, :] = primitive_neuron.forward(pos_vel_spikes)
        
with open('data/spike_motion', 'wb') as file:
        pickle.dump(spike_motion, file)